In [2]:
from scipy import stats
import anndata
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [3]:
adata=anndata.read("GTEx_8_tissues_snRNAseq_atlas_071421.public_obs.h5ad")

plist=[]
with  open  ("new_plist.txt",  "r")  as  myfile:
    myfile.readline()
    for line in myfile:
        plist.append(line.strip())
no_express=[]
for i in plist:
    if i not in adata.var_names:
        no_express.append(i)
        plist.remove(i)

In [4]:
## find pgenes that are expressed in low levels
sc.pp.filter_cells(adata,min_genes=1650)
sc.pp.filter_genes(adata,min_cells=50)
low_express=[]
for i in plist:
    if i not in adata.var_names:
        low_express.append(i)
for i in low_express:
        plist.remove(i)

In [5]:
def preprocess(adata):
    ## calculate mean and add to the dataframe
    X=pd.DataFrame(adata.X.todense())
    cell_name=adata.obs.index
    chr_name=adata.var.index
    X.index=adata.obs['tissue'].tolist()
    X.columns=chr_name
    mean=[]
    for gene in X.columns:
        mean.append(X[gene].mean())
    X.loc["mean"]=mean
    ## get pgenes that are expressed in low means
    X=X.sort_values(by="mean",axis=1)
    low_mean_q=X.iloc[len(X.index)-1,:].quantile(q=0.05,interpolation="lower")
    low_mean=[]
    for i in plist:
        if X[i]["mean"] < low_mean_q:
            low_mean.append(i)
            plist.remove(i)
    ## remove genes with mean lower than the 10th quantile
    index=X.loc["mean"].tolist().index(low_mean_q)
    X=X.iloc[:,index:]
    remove_list = []
    for i in plist:
        if i not in X.columns:
            remove_list.append(i)
    for i in remove_list:
        plist.remove(i)
    X.drop(labels="mean",axis=0, inplace=True)
    cell_type=adata.obs["Broad cell type"].tolist()
    X.insert(loc=0,column="Cell_type",value=cell_type)
    individual_type = adata.obs["Participant ID"].tolist()
    individual_type_list = list(set(individual_type))
    X.insert(loc=0, column="Individual_type", value = individual_type)
    return X, individual_type_list

In [6]:
X, individuals = preprocess(adata)


In [7]:
def get_individual_cv(X, individuals, cv):
    for individual in individuals:
        get_individual(X, individual, cv)

def get_individual(X, individual, cv):
    df = X[X["Individual_type"] == individual]
    tissue_name= df.index.tolist()
    tissue_name_unique = df.index.drop_duplicates().tolist()
    tissue_num = len(set(tissue_name))
    tissue_index, df = sort_by_cell(df, tissue_name, tissue_num)
    cell_index, cell_list, label_index, df = get_cell_label(df, tissue_num, tissue_index)
    tissue_type_dic={}
    for i in range(0,tissue_num):
        tissue_type_dic.update({tissue_name_unique[i]:{}})
        for j in range(label_index[i][0],label_index[i][1]+1):      
            tissue_type_dic[tissue_name_unique[i]].update({cell_list[j]:0})
    for gene in df.columns:
        if gene not in cv:
            cv.update({gene:{}})
        for tissue in tissue_name_unique:
            if tissue not in cv[gene]:
                cv[gene].update({tissue:{}})
            for key in tissue_type_dic[tissue].keys():
                if key not in cv[gene][tissue]:
                    cv[gene][tissue].update({key:[]})
    for i in range(0, tissue_num):
        get_cv(i, df, label_index, tissue_name_unique, cell_list, cell_index, cv)

    
        
def sort_by_cell(df, tissue_name, tissue_num):
    t_index=[]
    str=tissue_name[0]
    n=0
    m=0
    index=0
    for i in tissue_name:
        if (index==len(tissue_name)-1) or (index==0):
            m=index
            t_index.append(m)     
        elif i!=str:
            n=index
            m=index-1
            t_index.append(m)
            t_index.append(n) 
            str=i
        index+=1     
    for i in range(tissue_num):
        df.iloc[t_index[i*2]:t_index[i*2+1],:]=df.iloc[t_index[i*2]:t_index[i*2+1],:].sort_values(by="Cell_type")
    return t_index, df



def get_cell_label(df, tissue_num, t_index):
    cell_list=["0"]
    c_index=-1
    l_index=0
    cell_index=[]
    for i in df["Cell_type"].tolist():
        c_index+=1
        if i !=cell_list[l_index]:
            l_index+=1
            cell_list.append(i)
            cell_index.append(c_index)

    cell_list.remove("0")
    cell_index.append(len(df.index))
    df = df.drop("Cell_type",axis=1, inplace=False)
    df = df.drop("Individual_type",axis=1, inplace=False)
    # drop cell type with less than 20 samples
    n_cell_index=[]
    for i in range(len(cell_list)):
        n_cell_index.append([])
        n_cell_index[i].append(cell_index[i])
        n_cell_index[i].append(cell_index[i+1]-1)
    rmv_index=[]
    for i in range(len(cell_list)):
        if n_cell_index[i][1]-n_cell_index[i][0]<=20:
            rmv_index.append(i)
    for i in reversed(rmv_index):
        del n_cell_index[i]
        del cell_list[i]
    
    label_index=[]
    c_p=0
    for i in range(tissue_num):
        temp=[]
        temp.append(c_p)   
        for j in range(c_p, len(n_cell_index)):
            if(n_cell_index[j][0]>t_index[int(i)*2+1]):
                c_p=j
                temp.append(c_p-1)
                break
        if(int(i)==tissue_num-1):
            temp.append(len(n_cell_index)-1)
        label_index.append(temp)
    
    return n_cell_index, cell_list, label_index, df
    
    

def get_cv(tissue_index, X, label_index, tissue_type, cell_list, n_cell_index, gene_cv_dict):
    window_size = int(500)
    gene_list=X.columns.tolist()
    for i in range(label_index[tissue_index][0],label_index[tissue_index][1]+1):
        tissue = tissue_type[tissue_index]
        cell = cell_list[i]
        temp=pd.DataFrame(X.iloc[n_cell_index[i][0]:n_cell_index[i][1],:])
        temp.loc['mean'] = temp.apply(lambda x : x.sum())
        temp=temp.sort_values(by="mean",axis=1)
        cov=[]
        for gene in temp.columns:
            if(temp[gene]['mean']==0):
                cov.append(0)
            else:
                cov.append(temp[gene].std()/temp[gene]['mean'])
        temp.loc["cov"]=cov
        
        result={}
        for j in range(0,len(temp.columns)):
            window_keys_list=[]
            if(j<window_size/2):
                window_keys_list=temp.columns.tolist()[0:window_size]
            elif (window_size / 2) <= j < (len(temp.columns) - (window_size // 2)):
                window_keys_list=temp.columns.tolist()[(j - (window_size // 2)):(j + (window_size // 2))]
            else:
                window_keys_list=temp.columns.tolist()[len(temp.columns)-window_size//2:]
            cov_sort={}
            for key in window_keys_list:
                cov_sort[key]=temp[key]['cov']
            sorted_cov_key = sorted(cov_sort, key=cov_sort.get, reverse=False)
            gene_name=temp.columns.tolist()[j]
            result[gene_name] = ((sorted_cov_key.index(gene_name)+1.0)/window_size) * 100.0
            
        
        for gene in gene_list:
            gene_cv_dict[gene][tissue][cell].append(result[gene])
            

In [ ]:
cv = {}
get_individual_cv(X, individuals, cv)

In [ ]:
def save_variable(v,filename):
  f=open(filename,'wb')
  pickle.dump(v,f)
  f.close()
  return filename
 
def load_variable(filename):
  f=open(filename,'rb')
  r=pickle.load(f)
  f.close()
  return r

gene_cv_dict = save_variable(cv, 'sc_LCV.txt')